<a href="https://colab.research.google.com/github/maitysuvo19/internship_root2ai_classify_text/blob/main/Root2ai_classification_bagofwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [ ]:
import os
import pandas as pd
import numpy as np

#import feature extraction methods from sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words

#pre-processing of text
import string
import re

#import classifiers from sklearn
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

#import different metrics to evaluate the classifiers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report 
from sklearn import metrics

#import time function from time module to track the training duration
from time import time

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
df=pd.read_csv('/content/root2ai - Data.csv')

In [ ]:
df.head()

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22704 entries, 0 to 22703
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    22701 non-null  object
 1   Target  22704 non-null  object
dtypes: object(2)
memory usage: 354.9+ KB


In [ ]:
df["Target"].value_counts()/df.shape[0]

FinTech             0.376630
Cyber Security      0.116279
Bigdata             0.099850
Reg Tech            0.097163
credit reporting    0.076991
Blockchain          0.060562
Neobanks            0.047084
Microservices       0.043032
Stock Trading       0.034663
Robo Advising       0.032461
Data Security       0.015284
Name: Target, dtype: float64

In [ ]:
df['Target'] =df.Target.map({'FinTech':1, 'Cyber Security':2,'Bigdata':3,'Reg Tech':4,'credit reporting':5,'Blockchain':6,'Neobanks':7,'Microservices':8,'Stock Trading':9,'Robo Advising':10,'Data Security':11})
df = df[["Text","Target"]]
df.head()

,Text,Target
0,reserve bank forming expert committee based in...,6
1,director could play role financial system,6
2,preliminary discuss secure transaction study r...,6
3,security indeed prove essential transforming f...,6
4,bank settlement normally take three days based...,6


# Text Pre-processing

Typical steps involve tokenization, lower casing, removing, stop words, punctuation markers etc, and vectorization. Other processes such as stemming/lemmatization can also be performed. Here, we are performing the following steps: removing br tags, punctuation, numbers, and stopwords. While we are using sklearn's list of stopwords, there are several other stop word lists (e.g., from NLTK) or sometimes, custom stopword lists are needed depending on the task.

In [ ]:
stopwords = stop_words.ENGLISH_STOP_WORDS
def clean(doc): #doc is a string of text
    doc = doc.replace("</br>", " ") #This text contains a lot of <br/> tags.
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])
    #remove punctuation and numbers
    return doc

# Modeling

In [ ]:
#Step 1: train-test split
X = df.Text.fillna(' ') #the column text contains textual data to extract features from
y = df.Target #this is the column we are learning to predict. 
print(X.shape, y.shape)
# split X and y into training and testing sets. By default, it splits 75% training and 25% test
#random_state=1 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(22704,) (22704,)
(17028,) (17028,)
(5676,) (5676,)


In [ ]:
#Step 2-3: Preprocess and Vectorize train and test data
vect = CountVectorizer(preprocessor=clean) #instantiate a vectoriezer
X_train_dtm = vect.fit_transform(X_train)#use it to extract features from training data
#transform testing data (using training data's features)
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape, X_test_dtm.shape)
#i.e., the dimension of our feature vector is 31195!

(17028, 11329) (5676, 11329)


Naive Bayse Classifier

In [ ]:
#Step 3: Train the classifier and predict for test data
nb = MultinomialNB() #instantiate a Multinomial Naive Bayes model
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)#make class predictions for X_test_dtm

CPU times: user 15.5 ms, sys: 1.98 ms, total: 17.5 ms
Wall time: 20.6 ms


In [ ]:
#calculate evaluation measures:
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))

Accuracy:  0.6719520789288231
              precision    recall  f1-score   support

           1       0.59      0.92      0.72      2168
           2       0.63      0.54      0.58       655
           3       0.86      0.74      0.80       577
           4       0.90      0.80      0.85       532
           5       0.75      0.53      0.62       424
           6       0.80      0.35      0.49       355
           7       0.81      0.08      0.15       272
           8       0.93      0.50      0.65       225
           9       0.82      0.50      0.62       188
          10       0.83      0.14      0.23       185
          11       1.00      0.09      0.17        95

    accuracy                           0.67      5676
   macro avg       0.81      0.47      0.53      5676
weighted avg       0.72      0.67      0.64      5676



Logistic Regression Classifier

In [ ]:
logreg = LogisticRegression(class_weight="balanced") #instantiate a logistic regression model
logreg.fit(X_train_dtm, y_train) #fit the model with training data

#Make predictions on test data
y_pred_class = logreg.predict(X_test_dtm)

#calculate evaluation measures:
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))

Accuracy:  0.6005990133897111
              precision    recall  f1-score   support

           1       0.77      0.50      0.61      2168
           2       0.53      0.58      0.56       655
           3       0.63      0.83      0.71       577
           4       0.90      0.76      0.82       532
           5       0.60      0.67      0.63       424
           6       0.51      0.60      0.55       355
           7       0.31      0.56      0.39       272
           8       0.55      0.62      0.58       225
           9       0.43      0.66      0.52       188
          10       0.36      0.55      0.43       185
          11       0.32      0.39      0.35        95

    accuracy                           0.60      5676
   macro avg       0.54      0.61      0.56      5676
weighted avg       0.65      0.60      0.61      5676



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Support Vector Machine

In [ ]:
svm = LinearSVC(class_weight='balanced') #instantiate a support vector machine model
svm.fit(X_train_dtm, y_train) #fit the model with training data

#Make predictions on test data
y_pred_class = svm.predict(X_test_dtm)

#calculate evaluation measures:
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))

Accuracy:  0.6201550387596899
              precision    recall  f1-score   support

           1       0.71      0.61      0.66      2168
           2       0.55      0.59      0.57       655
           3       0.78      0.74      0.76       577
           4       0.89      0.78      0.83       532
           5       0.56      0.66      0.60       424
           6       0.52      0.55      0.53       355
           7       0.34      0.46      0.39       272
           8       0.60      0.59      0.59       225
           9       0.44      0.63      0.52       188
          10       0.33      0.49      0.40       185
          11       0.32      0.36      0.34        95

    accuracy                           0.62      5676
   macro avg       0.55      0.59      0.56      5676
weighted avg       0.64      0.62      0.63      5676



Our large feature vector could be creating a lot of noise in the form of very rarely occurring features that are not useful for learning. Let us change the count vectorizer to take a certain number of features as maximum.



In [ ]:
#Step 2-3: Preprocess and Vectorize train and test data
vect = CountVectorizer(preprocessor=clean,max_features=5000) #instantiate a vectoriezer
X_train_dtm = vect.fit_transform(X_train)#use it to extract features from training data
#transform testing data (using training data's features)
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape, X_test_dtm.shape)
#i.e., the dimension of our feature vector is 5000!

(17028, 5000) (5676, 5000)


Naive Bayse Classifier

In [ ]:
#Step 3: Train the classifier and predict for test data
nb = MultinomialNB() #instantiate a Multinomial Naive Bayes model
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)#make class predictions for X_test_dtm
#calculate evaluation measures:
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))

CPU times: user 14.8 ms, sys: 398 µs, total: 15.2 ms
Wall time: 15.1 ms
Accuracy:  0.6717758985200846
              precision    recall  f1-score   support

           1       0.63      0.85      0.72      2168
           2       0.58      0.57      0.58       655
           3       0.79      0.70      0.74       577
           4       0.87      0.77      0.82       532
           5       0.68      0.59      0.63       424
           6       0.73      0.49      0.59       355
           7       0.57      0.19      0.28       272
           8       0.82      0.63      0.71       225
           9       0.73      0.57      0.64       188
          10       0.61      0.24      0.35       185
          11       0.67      0.11      0.18        95

    accuracy                           0.67      5676
   macro avg       0.70      0.52      0.57      5676
weighted avg       0.68      0.67      0.66      5676



Logistic Regression Classifier

In [ ]:
logreg = LogisticRegression(class_weight="balanced") #instantiate a logistic regression model
logreg.fit(X_train_dtm, y_train) #fit the model with training data

#Make predictions on test data
y_pred_class = logreg.predict(X_test_dtm)

#calculate evaluation measures:
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))

Accuracy:  0.5863284002818887
              precision    recall  f1-score   support

           1       0.77      0.48      0.59      2168
           2       0.53      0.57      0.55       655
           3       0.59      0.82      0.69       577
           4       0.88      0.74      0.80       532
           5       0.60      0.66      0.63       424
           6       0.49      0.61      0.55       355
           7       0.30      0.55      0.39       272
           8       0.52      0.61      0.57       225
           9       0.43      0.67      0.53       188
          10       0.35      0.55      0.43       185
          11       0.29      0.39      0.33        95

    accuracy                           0.59      5676
   macro avg       0.52      0.60      0.55      5676
weighted avg       0.64      0.59      0.59      5676



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Support Vector Machine

In [ ]:
svm = LinearSVC(class_weight='balanced') #instantiate a support vector machine model
svm.fit(X_train_dtm, y_train) #fit the model with training data

#Make predictions on test data
y_pred_class = svm.predict(X_test_dtm)

#calculate evaluation measures:
print("Accuracy: ", accuracy_score(y_test, y_pred_class))
print(classification_report(y_test, y_pred_class))

Accuracy:  0.602184637068358
              precision    recall  f1-score   support

           1       0.75      0.56      0.64      2168
           2       0.55      0.58      0.56       655
           3       0.67      0.81      0.73       577
           4       0.86      0.77      0.81       532
           5       0.55      0.63      0.59       424
           6       0.50      0.54      0.52       355
           7       0.32      0.47      0.38       272
           8       0.51      0.56      0.54       225
           9       0.43      0.62      0.51       188
          10       0.31      0.47      0.38       185
          11       0.26      0.36      0.30        95

    accuracy                           0.60      5676
   macro avg       0.52      0.58      0.54      5676
weighted avg       0.63      0.60      0.61      5676

